In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
# Machine Learning: split, grid search e cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
pd.options.display.float_format= "{:.5f}".format
pd.options.display.max_colwidth = 5000

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Trabalho em grupo/Limpa_G4")
df = df.iloc[:,1:]
df

,Months since Last Donation,Number of Donations,Months since First Donation,Made Donation in March 2007
0,2,50,98,1
1,0,13,28,1
2,1,16,35,1
3,2,20,45,1
4,1,24,77,0
...,...,...,...,...
743,23,2,38,0
744,21,2,52,0
745,23,3,62,0
746,39,1,39,0


In [ ]:
# Separando X e y 
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

#Separando os dados em teste e treino

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify = y,  random_state= 6)

#Separando os dados de treinamento em treinamento (final) e validação

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify = y_train, random_state= 6)

# Calculando as proporções do valor 1 entre os splits
prop_real = df[df["Made Donation in March 2007"] == 1]["Made Donation in March 2007"].count()/df["Made Donation in March 2007"].count()
prop_treino = y_train[y_train == 1].count()/y_train.count()
prop_teste = y_test[y_test == 1].count()/y_test.count()
prop_validação = y_val[y_val == 1].count()/y_val.count()

# Criando um Data Frame para avaliar se a proporção de valor 1 (doou sangue em março de 2007) é identica nos datasets.
df_train_test_prop = pd.DataFrame([prop_real,prop_treino,prop_teste, prop_validação]).T
df_train_test_prop.rename(columns = {0:"Proporção Real", 1: "Proporção de Treinamento", 2: "Proporção de Teste", 3: "Proporção de Validação"}, 
                          inplace = True)

df_train_test_prop

,Proporção Real,Proporção de Treinamento,Proporção de Teste,Proporção de Validação
0,0.23797,0.23819,0.23894,0.23622


In [ ]:
# Normalizando X_train

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

# Normalizando o resto

X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Separando as métricas
métricas = ["accuracy", "precision", "recall", "f1_macro", "roc_auc"]
range(len(métricas))

range(0, 5)

In [ ]:
# Criando o modelo
scv = svm.SVC(random_state = 6)

# Separando os hiperparâmetros
hiperparametros = {"kernel": ["linear", "poly", "rbf", "sigmoid"],
                   "degree": [1,2,3,4,5],
                   "C": [10,1,0.1,0.05, 0.01],
                   "gamma": ["scale", "auto"]}

#Realizando o GridSearch
resultados = {}

for i in range(len(métricas)):
  
  GS = GridSearchCV(LR, hiperparametros, scoring = métricas[i])

  GS.fit(X_train, y_train)

  resultados[i] = pd.DataFrame(GS.cv_results_)


In [ ]:
# Renomeando o dicionário
resultados["accuracy"] = resultados.pop(0)
resultados["precision"] = resultados.pop(1)
resultados["recall"] = resultados.pop(2)
resultados["f1"] = resultados.pop(3)
resultados["roc_auc"] = resultados.pop(4)

In [ ]:
melhores_modelos = {"accuracy": resultados["accuracy"].sort_values("mean_test_score", ascending = False).iloc[0,8],
                    "precision": resultados["precision"].sort_values("mean_test_score", ascending = False).iloc[0,8],
                    "recall":resultados["recall"].sort_values("mean_test_score", ascending = False).iloc[0,8],
                    "f1": resultados["f1"].sort_values("mean_test_score", ascending = False).iloc[0,8],
                    "roc_auc": resultados["roc_auc"].sort_values("mean_test_score", ascending = False).iloc[0,8]}

In [ ]:
melhores_modelos

{'accuracy': {'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'},
 'f1': {'C': 1, 'degree': 3, 'gamma': 'scale', 'kernel': 'sigmoid'},
 'precision': {'C': 0.01, 'degree': 4, 'gamma': 'scale', 'kernel': 'poly'},
 'recall': {'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'},
 'roc_auc': {'C': 0.01, 'degree': 1, 'gamma': 'auto', 'kernel': 'poly'}}

In [ ]:
# Treinando e testando o SVM.scv com os "melhores" hiperparâmetros
SVC = svm.SVC(C = 1, 
              degree = 3, 
              gamma = "scale", 
              kernel = "sigmoid", 
              random_state = 6).fit(X_train, y_train)

# Prevendo y
y_pred = SVC.predict(X_val)

In [ ]:
# Separando as métricas
Acurácia = accuracy_score(y_val, y_pred)
Precisão =  precision_score(y_val, y_pred)
Recall = recall_score(y_val, y_pred)
F1 = f1_score(y_val, y_pred, average = "macro")
ROC = roc_auc_score(y_val, y_pred)
# Organizando as métricas em um DataFrame
resultados_val = pd.DataFrame({"Acurácia":Acurácia,
                                "Precisão": Precisão,
                                "Recall": Recall,
                                "F1": F1,
                                "ROC": ROC}, index = [0])

In [ ]:
resultados_val

,Acurácia,Precisão,Recall,F1,ROC
0,0.77165,0.52941,0.30000,0.62144,0.60876


In [ ]:
# Treinando e testando o SVM.scv com os "melhores" hiperparâmetros
SVC = svm.SVC(C = 10, 
              degree = 2, 
              gamma = "scale", 
              kernel = "sigmoid", 
              random_state = 6).fit(X_train, y_train)

# Prevendo y
y_pred = SVC.predict(X_val)

# Separando as métricas
Acurácia = accuracy_score(y_val, y_pred)
Precisão =  precision_score(y_val, y_pred)
Recall = recall_score(y_val, y_pred)
F1 = f1_score(y_val, y_pred, average = "macro")
ROC = roc_auc_score(y_val, y_pred)
# Organizando as métricas em um DataFrame
resultados_val = pd.DataFrame({"Acurácia":Acurácia,
                                "Precisão": Precisão,
                                "Recall": Recall,
                                "F1": F1,
                                "ROC": ROC}, index = [0])

resultados_val

,Acurácia,Precisão,Recall,F1,ROC
0,0.76378,0.50000,0.36667,0.63728,0.62663
